In [ ]:
%pip install 'vanna[openai]'

In [ ]:
from dotenv import load_dotenv
import os
from vanna.openai import OpenAI_Chat
from vanna.vannadb import VannaDB_VectorStore
import requests
import pandas as pd
import json

In [ ]:


# 加载 .env 文件中的环境变量
load_dotenv()

In [ ]:
class MyVanna(VannaDB_VectorStore, OpenAI_Chat):
    def __init__(self, config=None):
        MY_VANNA_MODEL = os.getenv('MY_VANNA_MODEL')
        VANNA_API_KEY = os.getenv('VANNA_API_KEY')
        VannaDB_VectorStore.__init__(self, vanna_model=MY_VANNA_MODEL, vanna_api_key=VANNA_API_KEY, config=config)
        OpenAI_Chat.__init__(self, config=config)

    def connect_to_custom_api(self, token: str, project: str):
        def run_sql_custom_api(sql: str) -> pd.DataFrame:
            url = 'http://dataapi.epian1.com/api/sql/query'
            params = {
                'token': token,
                'project': project
            }
            data = {
                'q': sql,
                'format': 'json'
            }
            response = requests.post(url, params=params, data=data)
            response.raise_for_status()  # 如果请求失败会抛出异常
            
            # 解析返回的 JSON 数据，每行是一个 JSON 对象
            data_lines = response.text.splitlines()
            data = [json.loads(line) for line in data_lines]
            
            # 将解析的数据转换为 DataFrame
            df = pd.DataFrame(data)
            return df

        self.dialect = "Custom API SQL"
        self.run_sql = run_sql_custom_api
        self.run_sql_is_set = True



vn = MyVanna(config={
    'api_key': os.getenv('OPENAI_API_KEY') , # 从环境变量中获取 API Key
    'model': 'gpt-4o'
})

vn.connect_to_custom_api(
    token=os.getenv('TOKEN'),
    project=os.getenv('PROJECT')
)

In [ ]:

# 运行 SQL 查询
sql_query = '''
SELECT date, event, distinct_id 
FROM events 
WHERE event = 'content_create' 
LIMIT 10
'''
test_data = vn.run_sql(sql_query)
print(test_data)

In [ ]:
from vanna.flask import VannaFlaskApp
app = VannaFlaskApp(vn)
app.run()